In [4]:
from debuging_ipynb import *

Pressure 109.30343371424011 kPa
Y :  [0.5448083332093433, 0.4551916667906567]
Pressure 92.15637652313664 kPa
X :  [0.16852941883435707, 0.831470581165643]
Temprature 103.30683592836805 C
Y :  [0.5421583358085414, 0.45784166419145855]
Temprature 109.13089105964352 C
X :  [0.1726275531656567, 0.8273724468343433]


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("Antonie_Constants.csv").drop("Unnamed: 0",axis=1)

In [7]:
df.head()

,formula,compound_name,A,B,C,Tmin,Tmax
0,CCL4,carbon-tetrachloride,6.89410,1219.580,227.170,-20,101.0
1,CCL3F,trichlorofluoromethane,6.88430,1043.010,236.860,-33,27.0
2,CCL2F2,dichlorodifluoromethane,6.68619,782.072,235.377,-119,-30.0
3,CCLF3,chlorotrifluoromethane,6.35109,522.061,231.677,-150,-81.0
4,CF4,carbon-tetrafluoride,6.97230,540.500,260.100,-180,-125.0


In [8]:
import random
from tqdm import tqdm

In [9]:
def random_dataset(num,df):
    count = 0
    num_rows = df.shape[0]
    l = []
    pbar = tqdm(total=num)
    while count != num:
        row1_index = random.randint(0, num_rows - 1)
        row2_index = random.randint(0,num_rows - 1)
        
        if row1_index != row2_index:
            Tmin = max(df["Tmin"][row1_index],df["Tmin"][row2_index])
            Tmax = min(df["Tmax"][row1_index],df["Tmax"][row2_index])
            
            if (Tmax - Tmin) > 0:
                randT = round(random.uniform(Tmin,Tmax),5)
                x1 = round(random.random(),5)
                x2 = 1 - x1
                
                d = {"x1" : x1, "x2" : x2, "T" : randT,
                     "A1" : df['A'][row1_index],
                     "B1" : df['B'][row1_index],
                     "C1" : df['C'][row1_index],
                     "A2" : df['A'][row2_index],
                     "B2" : df['B'][row2_index],
                     "C2" : df['C'][row2_index]}
                count += 1
                pbar.update(1)
                l.append(d)
    
    df = pd.DataFrame(l)
    return df        

In [10]:
ndf = random_dataset(500000,df)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499997/500000 [02:07<00:00, 4085.04it/s]

In [11]:
ndf.head()

,A1,A2,B1,B2,C1,C2,T,x1,x2
0,6.87015,5.51917,1197.60,460.684,225.00,164.594,-0.25001,0.13073,0.86927
1,6.88827,6.90252,1466.80,1368.720,203.00,207.440,49.34400,0.65543,0.34457
2,7.25390,7.21300,1507.41,1308.400,244.70,224.200,47.03561,0.82500,0.17500
3,6.89288,6.98834,1460.52,1647.100,201.90,201.000,161.48879,0.00849,0.99151
4,6.81616,6.86731,1486.62,1347.209,208.32,219.684,66.97549,0.35946,0.64054


In [12]:
tqdm.pandas()

In [13]:
ndf["BUBL_P"] = ndf.apply(lambda x: BUBL_P( [x["x1"], x["x2"]], x['T'], [ [x["A1"], x["B1"], x["C1"]], [x["A2"], x["B2"], x["C2"]]])[0],axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [03:14<00:00, 4085.04it/s]

In [14]:
ndf.head()

,A1,A2,B1,B2,C1,C2,T,x1,x2,BUBL_P
0,6.87015,5.51917,1197.60,460.684,225.00,164.594,-0.25001,0.13073,0.86927,13.753888
1,6.88827,6.90252,1466.80,1368.720,203.00,207.440,49.34400,0.65543,0.34457,3.581509
2,7.25390,7.21300,1507.41,1308.400,244.70,224.200,47.03561,0.82500,0.17500,8.557372
3,6.89288,6.98834,1460.52,1647.100,201.90,201.000,161.48879,0.00849,0.99151,11.576954
4,6.81616,6.86731,1486.62,1347.209,208.32,219.684,66.97549,0.35946,0.64054,7.078028


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test = train_test_split(ndf.drop('BUBL_P',axis=1),ndf['BUBL_P'],test_size = 0.001)

In [17]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

In [18]:
nn = MLPRegressor(hidden_layer_sizes=(10,10),max_iter=100,warm_start=True,verbose=True,learning_rate='adaptive')
nn.fit(X_train,y_train)
y_pred = nn.predict(X_test)
nn.score(X_test,y_test)

Iteration 1, loss = 247.89338402
Iteration 2, loss = 12.09935321
Iteration 3, loss = 9.58473185
Iteration 4, loss = 9.12926463
Iteration 5, loss = 8.08201150
Iteration 6, loss = 7.72576049
Iteration 7, loss = 7.54118524
Iteration 8, loss = 7.42911065
Iteration 9, loss = 7.30082117
Iteration 10, loss = 7.05058208
Iteration 11, loss = 6.37582460
Iteration 12, loss = 5.01312047
Iteration 13, loss = 4.41241780
Iteration 14, loss = 4.19174993
Iteration 15, loss = 4.06393117
Iteration 16, loss = 3.95714078
Iteration 17, loss = 3.82604968
Iteration 18, loss = 3.69168583
Iteration 19, loss = 3.53074074
Iteration 20, loss = 3.34499384
Iteration 21, loss = 3.17752274
Iteration 22, loss = 3.04205132
Iteration 23, loss = 2.88854898
Iteration 24, loss = 2.76838478
Iteration 25, loss = 2.72112813
Iteration 26, loss = 2.68644412
Iteration 27, loss = 2.64493927
Iteration 28, loss = 2.60020072
Iteration 29, loss = 2.57629233
Iteration 30, loss = 2.52291351
Iteration 31, loss = 2.47995921
Iteration 32, 

E:\programs\python\python 3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.9361095524092061

In [20]:
pd.DataFrame({'y':y_test,"y_pred":y_pred})

,y,y_pred
45301,11.975370,13.891110
8754,16.089213,16.320515
86546,6.918678,7.534758
311327,17.391845,16.553209
294915,13.418040,11.920404
482186,12.640059,13.475980
54476,12.929747,11.841426
65564,21.107093,21.562297
492023,17.442450,17.631029
415663,9.173198,8.841882


In [30]:
a = ndf.loc[0].drop("BUBL_P")
all_prams = [[a["A1"],a["B1"],a["C1"]],a["A2"],a["B2"],a["C2"]]
T = a['T']
X = [a['x1'],a['x2']]

In [32]:
%%timeit
BUBL_P(X,T,all_params)

14 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
DDD = pd.DataFrame(a).T
DDD

,A1,A2,B1,B2,C1,C2,T,x1,x2
0,6.87015,5.51917,1197.6,460.684,225.0,164.594,-0.25001,0.13073,0.86927


In [47]:
%%timeit
nn.predict(DDD)

489 µs ± 6.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
lr = LinearRegression()

In [23]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [24]:
y_pred = lr.predict(X_test)

In [26]:
lr.score(X_test,y_test)

0.4977642630378664

In [25]:
pd.DataFrame({'y':y_test,"y_pred":y_pred})

,y,y_pred
97492,3.186590,7.442498
42393,6.916559,6.919995
46884,8.536810,11.635727
1298,7.230263,10.396412
61955,11.301846,10.252109
26462,24.681650,13.178462
23481,16.688538,18.270885
94048,5.791201,6.910312
99940,13.773925,13.185399
49092,9.713561,12.093115
